This notebook reads in the preprocessed behavioral data from each participant creates one large dataframe with all reaction times organized by the type of action switch they represent (forage->forage, forage->check, check->check, check->forage). It also creates dataframes checkSeqRTs and forageSeqRTs which log the reaction times with respect to the position of each button press within a sequence of actions.

In [ ]:
import pandas as pd
import glob
import numpy as np
from scipy import stats
import os
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import chain
import scikit_posthocs as sp
from sklearn import preprocessing
import itertools
from sklearn.preprocessing import MinMaxScaler

drive = 'passport'  # 'passport' or 'seagate'

data_root = '/Users/haileytrier/Downloads/Trier_et_al_2023_code/data/'
to_include = ['sub103','sub105','sub109','sub111','sub117','sub119','sub121','sub123','sub125','sub127','sub129','sub131','sub204','sub208','sub212','sub214','sub216','sub218','sub222','sub224','sub228','sub230','sub232']

## Process data and extract RTs

In [9]:
RTs = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])
# Log sequences of forages that precede a check
nSeq = 7
forageSeqRTs = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])
# Log sequences of checks that precede a forage
checkSeqRTs = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])

def addRT(df, Phase:str=np.nan, ActionType:str=np.nan, TransitionType:str=np.nan,
        RT:float=np.nan, ParticipantID:str=np.nan, predatorTotal_currEnv:int=np.nan,
         envIndex:int=np.nan, timeInCurrEnv:float=np.nan, tpEpoch:float=np.nan,
          noOfCones:int=np.nan, predatorSpeed:float=np.nan,
         timePressure:float=np.nan,firstActionPostDisc:int=np.nan,rewAvail_currTP:float=np.nan,
         NForageBeforeCheck:int=np.nan,NCheckBeforeForage:int=np.nan):
    '''Add a row to the dataframe containing all RTs.'''
    df = pd.concat([df,pd.DataFrame(data={"Phase":[Phase],
                                          "ActionType":[ActionType],
                                          "TransitionType":[TransitionType],
                                          "RT":[RT],
                                          "ParticipantID":[ParticipantID],
                                          "envIndex":[envIndex],
                                          "predatorTotal_currEnv":[predatorTotal_currEnv],
                                          "timeInCurrEnv":[timeInCurrEnv],
                                          "tpEpoch":[tpEpoch],
                                          "noOfCones":[noOfCones],
                                          "predatorSpeed":[predatorSpeed],
                                          "timePressure":[timePressure],
                                          "firstActionPostDisc":[firstActionPostDisc],
                                          "rewAvail_currTP":[rewAvail_currTP],
                                          "NForageBeforeCheck":[NForageBeforeCheck],
                                          "NCheckBeforeForage":[NCheckBeforeForage]})])
    return df

def omitOutliers(df):
    '''
    Omit outliers based on 1.5x IQR. 
    df: Pandas dataframe with column RT.
    '''
    q25, q75 = np.percentile(df['RT'], 25), np.percentile(df['RT'], 75) # calculate interquartile range
    iqr = q75 - q25
    cut_off = iqr * 1.5 # calculate the outlier cutoff as 1.5x the interquartile range
    lower, upper = q25 - cut_off, q75 + cut_off
    df = df[(df['RT']>=lower) & (df['RT']<=upper)].copy()
    return df

allCheckBeforeFirstChecks = 0
allFirstChecks = 0

In [11]:
for subj_ID in to_include:

    dfm = pd.read_csv(data_root+subj_ID+'_preproExpBehav_postPD_unstandardized_withDisabled.csv')
    dfv = pd.read_csv(data_root+subj_ID+'_preproExpBehav_prePD_unstandardized_withDisabled.csv')

    # Merge pre- and post-discovery dataframes and sort by raw time stamp
    dfv = dfv.rename(columns={'ParticipantIDs':'ParticipantID'})
    dfm['phase'] = 'post'
    dfm['discovery'] = 0
    dfm['timePressure'] = np.nan
    dfm['firstCheck_corrected'] = 1
    dfv['phase'] = 'pre'
    dfv = dfv.reset_index(drop=True)
    dfm = dfm.reset_index(drop=True)
    cols = ['buttonID', 'buttonName','msec_since_tenth_pulse','tpEpoch','timeInCurrEnv',
            'envIndex','predatorTotal_currEnv','discovery','ParticipantID','phase','predatorSpeed',
            'noOfCones', 'timePressure','firstCheck_corrected','rewAvail_currTP']
    df = pd.concat([dfv.loc[:,cols], dfm.loc[:,cols]], ignore_index=True)
    df = df.sort_values(by=['msec_since_tenth_pulse'])
    df = df.reset_index(drop=True)
    del dfv, dfm, cols
    
    # Omit any rare errors
    df = df.drop(df[df.timeInCurrEnv>90000].index).reset_index(drop=True)
    df.drop(df[df['envIndex'] > 26].index, inplace = True)
    
    # Make new variables
    df.insert(0,"prevButtonName",df.buttonName.shift(periods=1))
    df = df.assign(**{'firstForages':0, 'subseqForages':0, 'subseqChecks':0}).copy()

    # Note first & subsequent forages
    df.loc[(df['buttonName']=='forage') & (df['prevButtonName'] != 'forage'),'firstForages'] = 1
    firstEnvForages = [x for x in range(1,len(df)) if df.envIndex[x-1]!=df.envIndex[x] if df.buttonName[x]=='forage'] # first forage in the block
    if len(firstEnvForages) != 0: df.loc[firstEnvForages,'firstForages'] = 1
    df.loc[(df['buttonName']=='forage') & (df['firstForages']==0),'subseqForages'] = 1 # All other forages are subsequent
    del firstEnvForages
    
    # Catch first checks that happen at start of epoch
    exception_firstChecks = [x for x in range(1,len(df)) if df.predatorTotal_currEnv[x-1]!=df.predatorTotal_currEnv[x] if 'vigilence' in df.buttonName[x]]
    if len(exception_firstChecks)>0: df.loc[exception_firstChecks,'firstCheck_corrected'] = 1
    del exception_firstChecks
    
    # Any check in a block with 1 direction for checking is a first check
    firstCheckOneDir = [x for x in range(1, len(df)) if df.noOfCones[x]==1 if 'vigilence' in df.buttonName[x]]
    if len(firstCheckOneDir)>0: df.loc[firstCheckOneDir,'firstCheck_corrected'] = 1
    del firstCheckOneDir
    
    # One-hot for subsequent checks
    subseqChecksIndices = [x for x in range(len(df)) if ('vigilence' in df.loc[x,'buttonName'] and df.loc[x,'firstCheck_corrected']==0)]
    df.loc[subseqChecksIndices, 'subseqChecks'] = 1
    del subseqChecksIndices
    
    with warnings.catch_warnings():
        warnings.simplefilter('error')

    ##########################################################################
    # Get RTs
    for b in df['envIndex'].unique(): # iterate through blocks
        for e in df['predatorTotal_currEnv'].unique(): # iterate through epochs
            temp_pre = df.loc[(df.predatorTotal_currEnv==e)&(df.envIndex==b)&(df.phase=='pre'),:].reset_index()
            temp_post = df.loc[(df.predatorTotal_currEnv==e)&(df.envIndex==b)&(df.phase=='post'),:].reset_index()
    
            ######## Pre-discovery phase ########
            # Checks preceded by forage
            forageBeforeCheckInds = [x for x in range(1,len(temp_pre)) if ('vigilence' in temp_pre.loc[x,'buttonName']) and (temp_pre.loc[x,'prevButtonName']=='forage')]
            for x in forageBeforeCheckInds:
                RT = (temp_pre.loc[x,'tpEpoch']-temp_pre.loc[x-1,'tpEpoch'])-1500
                RTs = addRT(RTs,Phase='Pre',
                            ActionType='check',
                            TransitionType='Forage-to-check',
                            RT=RT,
                            ParticipantID=subj_ID,
                            envIndex=temp_pre.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_pre.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_pre.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_pre.loc[x,'tpEpoch'],
                            noOfCones=temp_pre.loc[x,'noOfCones'],
                            predatorSpeed=temp_pre.loc[x,'predatorSpeed'],
                            timePressure=temp_pre.loc[x,'timePressure'],
                            rewAvail_currTP=temp_pre.loc[x,'rewAvail_currTP'])
                ind = x-1
                prevActions = 1
                while ind>0 and temp_pre.loc[ind,'prevButtonName']=='forage' and prevActions < nSeq:
                    forageSeqRTs = addRT(forageSeqRTs, Phase='Pre', ActionType='forage', 
                                TransitionType='Forage-to-forage',
                                NForageBeforeCheck=prevActions, 
                                RT=(temp_pre.loc[ind,'tpEpoch']-temp_pre.loc[ind-1,'tpEpoch'])-1500, 
                                ParticipantID=subj_ID,
                                envIndex=temp_pre.loc[ind,'envIndex'],
                                predatorTotal_currEnv=temp_pre.loc[ind,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_pre.loc[ind,'timeInCurrEnv'],
                                noOfCones=temp_pre.loc[ind,'noOfCones'],
                                predatorSpeed=temp_pre.loc[ind,'predatorSpeed'],
                                timePressure=temp_pre.loc[ind,'timePressure'],
                                rewAvail_currTP=temp_pre.loc[ind,'rewAvail_currTP'])
                    ind -= 1
                    prevActions += 1
            del forageBeforeCheckInds

            # Count how many first checks are preceded by a check (for stats)
            checkBeforeFirstCheckInds = [x for x in range(1,len(temp_pre)) if ('vigilence' in temp_pre.loc[x,'prevButtonName']) and (temp_pre.loc[x,'firstCheck_corrected']==1)]
            allCheckBeforeFirstChecks += len(checkBeforeFirstCheckInds)    
            firstCheckInds = [x for x in range(1,len(temp_pre)) if (temp_pre.loc[x,'firstCheck_corrected']==1)]
            allFirstChecks += len(firstCheckInds)

            # Forages preceded by a check
            checkBeforeForageInds = [x for x in range(1,len(temp_pre)) if ('vigilence' in temp_pre.loc[x,'prevButtonName']) and (temp_pre.loc[x,'buttonName']=='forage')]
            for x in checkBeforeForageInds:
                RT = (temp_pre.loc[x,'tpEpoch']-temp_pre.loc[x-1,'tpEpoch'])-500
                RTs = addRT(RTs,Phase='Pre',ActionType='forage',TransitionType='Check-to-forage',RT=RT, 
                            ParticipantID=subj_ID,envIndex=temp_pre.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_pre.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_pre.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_pre.loc[x,'tpEpoch'],
                            noOfCones=temp_pre.loc[x,'noOfCones'],
                            predatorSpeed=temp_pre.loc[x,'predatorSpeed'],
                            timePressure=temp_pre.loc[x,'timePressure'],
                            rewAvail_currTP=temp_pre.loc[x,'rewAvail_currTP'])
                ind = x-1
                prevActions = 1
                while ind>0 and 'vigilence' in temp_pre.loc[ind,'prevButtonName'] and prevActions < 4:
                    checkSeqRTs = addRT(checkSeqRTs, Phase='Pre', ActionType='check',
                                TransitionType='Check-to-check',
                                NCheckBeforeForage=prevActions, 
                                RT=(temp_pre.loc[ind,'tpEpoch']-temp_pre.loc[ind-1,'tpEpoch'])-500, 
                                ParticipantID=subj_ID,
                                envIndex=temp_pre.loc[ind,'envIndex'],
                                predatorTotal_currEnv=temp_pre.loc[ind,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_pre.loc[ind,'timeInCurrEnv'],
                                noOfCones=temp_pre.loc[ind,'noOfCones'],
                                predatorSpeed=temp_pre.loc[ind,'predatorSpeed'],
                                timePressure=temp_pre.loc[ind,'timePressure'],
                                rewAvail_currTP=temp_pre.loc[ind,'rewAvail_currTP'])
                    ind -= 1
                    prevActions += 1
                    
            del checkBeforeForageInds

            # Subsequent checks
            checkBeforeCheckInds = [x for x in range(1,len(temp_pre)) if ('vigilence' in temp_pre.loc[x,'buttonName']) and ('vigilence' in temp_pre.loc[x,'prevButtonName'])]
            for x in checkBeforeCheckInds:
                RT = (temp_pre.loc[x,'tpEpoch']-temp_pre.loc[x-1,'tpEpoch'])-500
                RTs = addRT(RTs,Phase='Pre',ActionType='check',TransitionType='Check-to-check',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_pre.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_pre.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_pre.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_pre.loc[x,'tpEpoch'],
                            noOfCones=temp_pre.loc[x,'noOfCones'],
                            predatorSpeed=temp_pre.loc[x,'predatorSpeed'],
                            timePressure=temp_pre.loc[x,'timePressure'],
                            rewAvail_currTP=temp_pre.loc[x,'rewAvail_currTP'])
            del checkBeforeCheckInds
            
            # Subsequent forages
            forageBeforeForageInds = [x for x in range(1,len(temp_pre)) if (temp_pre.loc[x,'buttonName']=='forage') and (temp_pre.loc[x,'prevButtonName']=='forage')]
            for x in forageBeforeForageInds:
                RT = (temp_pre.loc[x,'tpEpoch']-temp_pre.loc[x-1,'tpEpoch'])-1500
                RTs = addRT(RTs,Phase='Pre',ActionType='forage',
                            TransitionType='Forage-to-forage',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_pre.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_pre.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_pre.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_pre.loc[x,'tpEpoch'],
                            noOfCones=temp_pre.loc[x,'noOfCones'],
                            predatorSpeed=temp_pre.loc[x,'predatorSpeed'],
                            timePressure=temp_pre.loc[x,'timePressure'],
                            rewAvail_currTP=temp_pre.loc[x,'rewAvail_currTP'])
            del forageBeforeForageInds
            
            ######## Post-discovery phase ########
            # Checks preceded by a forage
            forageBeforeCheckInds = [x for x in range(1,len(temp_post)) if ('vigilence' in temp_post.loc[x,'buttonName']) and (temp_post.loc[x-1,'buttonName']=='forage')]
            for x in forageBeforeCheckInds:
                RT = (temp_post.loc[x,'tpEpoch']-temp_post.loc[x-1,'tpEpoch'])-1500
                RTs = addRT(RTs,Phase='Post',ActionType='check',TransitionType='Forage-to-check',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_post.loc[x,'tpEpoch'],
                            noOfCones=temp_post.loc[x,'noOfCones'],
                            predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                            rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
                ind = x-1
                prevActions = 1
                while ind>0 and temp_post.loc[ind-1,'buttonName']=='forage' and prevActions < nSeq:
                    forageSeqRTs = addRT(forageSeqRTs, Phase='Post', ActionType='forage', 
                                TransitionType='Forage-to-forage',
                                NForageBeforeCheck=prevActions, 
                                RT=(temp_post.loc[ind,'tpEpoch']-temp_post.loc[ind-1,'tpEpoch'])-1500, 
                                ParticipantID=subj_ID,
                                envIndex=temp_post.loc[ind,'envIndex'],
                                predatorTotal_currEnv=temp_post.loc[ind,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_post.loc[ind,'timeInCurrEnv'],
                                noOfCones=temp_post.loc[ind,'noOfCones'],
                                predatorSpeed=temp_post.loc[ind,'predatorSpeed'],
                                timePressure=temp_post.loc[ind,'timePressure'],
                                rewAvail_currTP=temp_post.loc[ind,'rewAvail_currTP'])
                    ind -= 1
                    prevActions += 1
            del forageBeforeCheckInds
            
            # Forages preceded by a check
            checkBeforeForageInds = [x for x in range(1,len(temp_post)) if ('vigilence' in temp_post.loc[x-1,'buttonName']) and (temp_post.loc[x,'buttonName']=='forage')]
            for x in checkBeforeForageInds:
                RT = (temp_post.loc[x,'tpEpoch']-temp_post.loc[x-1,'tpEpoch'])-500
                RTs = addRT(RTs,Phase='Post',ActionType='forage',TransitionType='Check-to-forage',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_post.loc[x,'tpEpoch'],
                            noOfCones=temp_post.loc[x,'noOfCones'],
                            predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                            rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
            del checkBeforeForageInds
            
            # Hides
            hideInds = [x for x in range(1,len(temp_post)) if temp_post.loc[x,'buttonName']=='hide']
            for x in hideInds:
                RT = temp_post.loc[x,'tpEpoch']-temp_post.loc[x-1,'tpEpoch']
                if temp_post.loc[x-1,'buttonName']=='forage':
                    RTs = addRT(RTs,Phase='Post',ActionType='Hide',TransitionType='Forage-to-hide',RT=RT-1500,
                                ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                                predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                                tpEpoch=temp_post.loc[x,'tpEpoch'],
                                noOfCones=temp_post.loc[x,'noOfCones'],
                            predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                            rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
                elif 'vigilence' in temp_post.loc[x-1,'buttonName']:
                    RTs = addRT(RTs,Phase='Post',ActionType='Hide',TransitionType='Check-to-hide',RT=RT-500,
                                ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                                predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                                tpEpoch=temp_post.loc[x,'tpEpoch'],
                                noOfCones=temp_post.loc[x,'noOfCones'],
                                predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                                rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
            del hideInds
            
            # Subsequent checks (preceded by check)
            checkBeforeCheckInds = [x for x in range(1,len(temp_post)) if ('vigilence' in temp_post.loc[x,'buttonName']) and ('vigilence' in temp_post.loc[x-1,'buttonName'])]
            for x in checkBeforeCheckInds:
                RT = (temp_post.loc[x,'tpEpoch']-temp_post.loc[x-1,'tpEpoch'])-500
                RTs = addRT(RTs,Phase='Post',ActionType='check',TransitionType='Check-to-check',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_post.loc[x,'tpEpoch'],
                            noOfCones=temp_post.loc[x,'noOfCones'],
                            predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                            rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
            del checkBeforeCheckInds
            
            # Subsequent forages (preceded by forage)
            forageBeforeForageInds = [x for x in range(1,len(temp_post)) if (temp_post.loc[x,'buttonName']=='forage') and (temp_post.loc[x-1,'buttonName']=='forage')]
            for x in forageBeforeForageInds:
                RT = (temp_post.loc[x,'tpEpoch']-temp_post.loc[x-1,'tpEpoch'])-1500
                RTs = addRT(RTs,Phase='Post',ActionType='forage',TransitionType='Forage-to-forage',RT=RT,
                            ParticipantID=subj_ID,envIndex=temp_post.loc[x,'envIndex'],
                            predatorTotal_currEnv=temp_post.loc[x,'predatorTotal_currEnv'],
                            timeInCurrEnv=temp_post.loc[x,'timeInCurrEnv'],
                            tpEpoch=temp_post.loc[x,'tpEpoch'],
                            noOfCones=temp_post.loc[x,'noOfCones'],
                            predatorSpeed=temp_post.loc[x,'predatorSpeed'],
                            rewAvail_currTP=temp_post.loc[x,'rewAvail_currTP'])
            del forageBeforeForageInds
            
            ######## Post-discovery RT ########
            if (len(temp_pre)>0) and (len(temp_post)>0):
                RT = temp_post.loc[0,'tpEpoch'] - temp_pre.loc[len(temp_pre)-1,'tpEpoch'] - 500
                atype = 'check' if 'vigilence' in temp_post.loc[0,'buttonName'] else temp_post.loc[0,'buttonName']
                ttype = 'Check-to-'+atype
                RTs = addRT(RTs,Phase='Post',ActionType=atype,
                                TransitionType=ttype,RT=RT,
                                ParticipantID=subj_ID,envIndex=temp_post.loc[0,'envIndex'],
                                predatorTotal_currEnv=temp_post.loc[0,'predatorTotal_currEnv'],
                                timeInCurrEnv=temp_post.loc[0,'timeInCurrEnv'],
                                tpEpoch=temp_post.loc[0,'tpEpoch'],
                                noOfCones=temp_post.loc[0,'noOfCones'],
                                predatorSpeed=temp_post.loc[0,'predatorSpeed'],
                                firstActionPostDisc=1,
                                rewAvail_currTP=temp_post.loc[0,'rewAvail_currTP'])
                
del temp_post, temp_pre, df, ttype, atype

/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/135356633.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={"Phase":[Phase],
/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/135356633.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,pd.DataFrame(data={"Phase":[Phase],
/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/135356633.py:24: FutureWarning: The behavior of DataFrame concatenati

In [12]:
# Further clean data:
# Get rid of any response where the button press happened erroneously fast (a rare technical error)
RTs = RTs[RTs['RT']>0].copy()
RTs = RTs.reset_index(drop=True)

forageSeqRTs = forageSeqRTs[forageSeqRTs['RT']>0].copy()
forageSeqRTs = forageSeqRTs.reset_index(drop=True)

checkSeqRTs = checkSeqRTs[checkSeqRTs['RT']>0].copy()
checkSeqRTs = checkSeqRTs.reset_index(drop=True)

In [13]:
# Outlier removal across all participants:

RTs = omitOutliers(RTs)
forageSeqRTs = omitOutliers(forageSeqRTs)
checkSeqRTs = omitOutliers(checkSeqRTs)

In [15]:
# Instead of standardizing, scale to between 0-n within-participant to be consistent with RT data type
# O-1 range will be used to visualize the standardized RTs.
# 0-600ms range will be used for modelling because it works best with the shifted lognormal family type which 
# expects non-standardized values.
n=1
forageSeqRTs2 = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])
RTs2 = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])
checkSeqRTs2 = pd.DataFrame(columns=["Phase","ActionType","TransitionType",
                            "RT","ParticipantID","envIndex","predatorTotal_currEnv",
                            "timeInCurrEnv","tpEpoch","noOfCones","predatorSpeed","timePressure",
                            "rewAvail_currTP","firstActionPostDisc","NForageBeforeCheck","NCheckBeforeForage"])

for subj_ID in to_include:
    
    temp = forageSeqRTs.loc[forageSeqRTs.ParticipantID==subj_ID,:].copy()
    temp.RT = MinMaxScaler(feature_range=(0, n)).fit_transform(np.atleast_2d(temp.RT).T)
    forageSeqRTs2 = pd.concat([forageSeqRTs2, temp])
    del temp
    
    temp = RTs.loc[RTs.ParticipantID==subj_ID,:].copy()
    temp.RT = MinMaxScaler(feature_range=(0, n)).fit_transform(np.atleast_2d(temp.RT).T)
    RTs2 = pd.concat([RTs2, temp])
    del temp
    
    temp = checkSeqRTs.loc[checkSeqRTs.ParticipantID==subj_ID,:].copy()
    temp.RT = MinMaxScaler(feature_range=(0, n)).fit_transform(np.atleast_2d(temp.RT).T)
    checkSeqRTs2 = pd.concat([checkSeqRTs2, temp])
    del temp


/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/888319642.py:23: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forageSeqRTs2 = pd.concat([forageSeqRTs2, temp])
/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/888319642.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  RTs2 = pd.concat([RTs2, temp])
/var/folders/ck/bj1v04rj5k97nqvzmysnm2wr0000gn/T/ipykernel_60923/888319642.py:33: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entrie

In [ ]:
# Save
# RTs2.to_csv(data_root+'RTs_OverallOutlierOmit_withDisabled_minMaxTrans0-1_reproduced.csv')
# forageSeqRTs2.to_csv(data_root+'forageSeqRTs_OverallOutlierOmit_withDisabled_minMaxTrans0-1_reproduced.csv')
# checkSeqRTs.to_csv(data_root+'checkSeqRTs_OverallOutlierOmit_withDisabled_minMaxTrans0-1_reproduced.csv')